In [82]:
#import packages
import pandas as pd
import numpy as np 

#Imports for plotting
from ast import literal_eval 
import pickle
import seaborn as sns
import matplotlib.pyplot as plt   
from matplotlib import cm
import matplotlib_venn as venn
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles
%matplotlib inline

#options
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
pd.options.display.max_rows = 50
import warnings
warnings.filterwarnings('ignore') 

#NLP Packages
from textwrap import wrap
from textblob import TextBlob 
from wordcloud import WordCloud, ImageColorGenerator 
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.probability import FreqDist 
from nltk import RegexpTokenizer, PorterStemmer, pos_tag  
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import string
import re  

from sklearn.model_selection import train_test_split 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #WBOT
from sklearn.feature_extraction.text import TfidfTransformer


nltk.download('averaged_perceptron_tagger')  

# sklearn
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

# xgboost
from xgboost import XGBClassifier

# utilities
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# preferences
pd.set_option('display.max_columns', None)
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ji/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [121]:
#Load data
df = pd.read_pickle('df_processed_full.pickle')

#Preview dataset
df.head(1)

,rating_overall,employee_title_title_known,employee_title_title_unknown,employee_status_current_employee,employee_status_former_employee,employee_status_unknown_status,helpful_helpful,helpful_not_helpful,recommends_does_not_recommend,recommends_recommends,recommends_unknown_recommends,area_central,area_east_coast,area_mountain,area_outside_US,area_unknown,area_west_coast,company_outlook_negative_outlook,company_outlook_neutral_outlook,company_outlook_outlook_unknown,company_outlook_positive_outlook,opinion_of_CEO_approves_of_CEO,opinion_of_CEO_disapproves_of_CEO,opinion_of_CEO_no_opinion_of_CEO,opinion_of_CEO_unknown_opinion_of_CEO,review_text,orig_text,lem,stem,pos,noun,adj,adv,verb,characters,tokens,words,sentences,avg_word_len,avg_sent_len,stem_str,sentiment
0,5.0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,"[great, always, room, improvement, everybody, works, together, cons, love, job, helpfu]",great always room for improvement. everybody works together. no cons i love my job Helpfu nan,"[great, always, room, improvement, everybody, work, together, con, love, job, helpfu]","[great, alway, room, improv, everybodi, work, togeth, con, love, job, helpfu]","[JJ, RB, NN, NN, NN, VBZ, RB, NNS, VBP, NN, NN]",0.545455,0.090909,0.181818,0.181818,94,11,17,3,5.529412,5.666667,great alway room improv everybodi work togeth con love job helpfu,0.65


In [122]:
df.keys()

Index(['rating_overall', 'employee_title_title_known',
       'employee_title_title_unknown', 'employee_status_current_employee',
       'employee_status_former_employee', 'employee_status_unknown_status',
       'helpful_helpful', 'helpful_not_helpful',
       'recommends_does_not_recommend', 'recommends_recommends',
       'recommends_unknown_recommends', 'area_central', 'area_east_coast',
       'area_mountain', 'area_outside_US', 'area_unknown', 'area_west_coast',
       'company_outlook_negative_outlook', 'company_outlook_neutral_outlook',
       'company_outlook_outlook_unknown', 'company_outlook_positive_outlook',
       'opinion_of_CEO_approves_of_CEO', 'opinion_of_CEO_disapproves_of_CEO',
       'opinion_of_CEO_no_opinion_of_CEO',
       'opinion_of_CEO_unknown_opinion_of_CEO', 'review_text', 'orig_text',
       'lem', 'stem', 'pos', 'noun', 'adj', 'adv', 'verb', 'characters',
       'tokens', 'words', 'sentences', 'avg_word_len', 'avg_sent_len',
       'stem_str', 'sentiment'

In [135]:
needed = ['rating_overall', 'employee_title_title_known','employee_title_title_unknown', 'employee_status_current_employee', 
          'employee_status_former_employee', 'employee_status_unknown_status', 'helpful_helpful', 'helpful_not_helpful','recommends_does_not_recommend',
          'recommends_recommends','recommends_unknown_recommends', 'area_central', 'area_east_coast','area_mountain', 'area_outside_US', 
          'area_unknown', 'area_west_coast','company_outlook_negative_outlook', 'company_outlook_neutral_outlook',
          'company_outlook_outlook_unknown', 'company_outlook_positive_outlook', 'opinion_of_CEO_approves_of_CEO', 
          'opinion_of_CEO_disapproves_of_CEO', 'opinion_of_CEO_no_opinion_of_CEO','opinion_of_CEO_unknown_opinion_of_CEO', 
           'noun', 'adj', 'adv', 'verb', 'characters','tokens', 'words', 'sentences', 'avg_word_len', 'avg_sent_len', 'stem_str', 'sentiment']

dfa = df[needed] 

In [136]:
# seperate target from features 

X = dfa.drop(columns=['rating_overall'])
y = df['rating_overall']

# splitting before vectorizing:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)

In [137]:
print(X_train.shape) 
print(X_test.shape)

(11940, 36)
(2986, 36)


In [138]:
vectorizer = TfidfVectorizer(ngram_range=(2, 3),  # include uni and 3-rams only
                            max_df=.999, min_df=.007) 

In [139]:
tfidf_data_train = vectorizer.fit_transform(X_train['stem_str'])
tfidf_data_test = vectorizer.transform(X_test['stem_str'])
# tfidf_data_test.data

In [140]:
print(tfidf_data_train.shape) 
print(tfidf_data_test.shape)

(11940, 188)
(2986, 188)


In [141]:
X_train = X_train.drop(columns=['stem_str'])
X_test = X_test.drop(columns=['stem_str'])

In [142]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11940 entries, 6078 to 5327
Data columns (total 35 columns):
employee_title_title_known               11940 non-null int64
employee_title_title_unknown             11940 non-null int64
employee_status_current_employee         11940 non-null int64
employee_status_former_employee          11940 non-null int64
employee_status_unknown_status           11940 non-null int64
helpful_helpful                          11940 non-null int64
helpful_not_helpful                      11940 non-null int64
recommends_does_not_recommend            11940 non-null int64
recommends_recommends                    11940 non-null int64
recommends_unknown_recommends            11940 non-null int64
area_central                             11940 non-null int64
area_east_coast                          11940 non-null int64
area_mountain                            11940 non-null int64
area_outside_US                          11940 non-null int64
area_unknown         

In [143]:
non_zero_cols = tfidf_data_train.nnz / float(tfidf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tfidf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 3.129899497487437
Percentage of columns containing 0: 0.98335159841762


In [144]:
# convert tfidf to array and concatenate train and test sets

X_train = pd.concat([pd.DataFrame(tfidf_data_train.toarray()), X_train.reset_index()], axis = 1)
X_test = pd.concat([pd.DataFrame(tfidf_data_test.toarray()), X_test.reset_index()], axis = 1)

In [102]:
# X_train['index']

0         6078
1        10361
2        10669
3         8475
4         1716
         ...  
11935    14704
11936       48
11937     8964
11938     5944
11939     5327
Name: index, Length: 11940, dtype: int64

In [145]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=32)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=32, verbose=0,
                       warm_start=False)

In [146]:
rf_test_preds = rf_classifier.predict(X_test)

In [147]:
rf_recall = recall_score(y_test, rf_test_preds, average='weighted')
rf_acc_score = accuracy_score(y_test, rf_test_preds)
rf_f1_score = f1_score(y_test, rf_test_preds, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(rf_recall))
print("Testing Accuracy: {:.4}".format(rf_acc_score))
print("F1 Score: {:.4}".format(rf_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.6072
Testing Accuracy: 0.6072
F1 Score: 0.5894


In [148]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [149]:
xgb_test_preds = xgb.predict(X_test)

In [150]:
xgb_recall = recall_score(y_test, xgb_test_preds, average='weighted')
xgb_acc_score = accuracy_score(y_test, xgb_test_preds)
xgb_f1_score = f1_score(y_test, xgb_test_preds, average='weighted')

print('XGBOOST with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(xgb_recall))

print("Testing Accuracy: {:.4}".format(xgb_acc_score))
print("F1 Score: {:.4}".format(xgb_f1_score))

XGBOOST with Stematized Features:
---------------
Recall: 0.6125
Testing Accuracy: 0.6125
F1 Score: 0.5968


In [151]:
#scale X_train and X_test for svm

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [152]:
svm_clsf = SVC()

In [153]:
def modifier (text):
    if '1' in text: return 1
    if '2' in text: return 2
    if '3' in text: return 3
    if '4' in text: return 4
    if '5' in text: return 5

In [154]:
# y_train = y_train.map(modifier)
y_test = y_test.map(modifier)

TypeError: argument of type 'float' is not iterable

In [155]:
svm_clsf.fit(scaled_X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [156]:
svm_test_preds = svm_clsf.predict(scaled_X_test)

In [157]:
svm_recall = recall_score(y_test, svm_test_preds, average='weighted')
svm_acc_score = accuracy_score(y_test, svm_test_preds)
svm_f1_score = f1_score(y_test, svm_test_preds, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(svm_recall))

print("Testing Accuracy: {:.4}".format(svm_acc_score))
print("F1 Score: {:.4}".format(svm_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.5633
Testing Accuracy: 0.5633
F1 Score: 0.5381


In [158]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train, y_train)

In [159]:
log_test_pred = model_log.predict(X_test)

In [160]:
log_recall = recall_score(y_test, log_test_pred, average='weighted')
log_acc_score = accuracy_score(y_test, log_test_pred)
log_f1_score = f1_score(y_test, log_test_pred, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(log_recall))

print("Testing Accuracy: {:.4}".format(log_acc_score))
print("F1 Score: {:.4}".format(log_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.5727
Testing Accuracy: 0.5727
F1 Score: 0.5458
